# Riot API로 받지 못하는 정보 OP.GG에서 크롤링하기

In [21]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

### 각 라인의 탑 티어 챔피언 구하기

In [22]:
url = 'https://www.op.gg/champion/statistics'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

- 필요한 함수들

In [23]:
# 챔피언, 아이템, 스펠, 룬 데이터
champion_df = pd.read_csv('../00. data/champions.csv', index_col = 0)
champion_df['lower_name'] = champion_df['eng_name'].apply(lambda r : r.lower()) # OP.GG의 챔피언 찾기를 위해 소문자화
item_df = pd.read_csv('../00. data/items.csv', index_col = 0)
spell_df = pd.read_csv('../00. data/spell.csv', index_col = 0)
rune_df = pd.read_csv('../00. data/runes.csv', index_col = 0)

In [24]:
# 챔피언의 영문이름을 한글로 바꿔주는 함수
def translation_champion(eng):
    try :
        if eng == 'Wukong' :
            champion = '오공'
        elif eng == 'Nunu &amp; Willump':
            champion = '누누와 월럼프'
        else :
            eng = eng.lower().replace(' ','').replace('.','')
            champion = champion_df['name'][champion_df['name'][champion_df['lower_name'] == eng].index[0]]
    except:
        champion = eng 
    return champion

In [25]:
# 아이템키로 아이템 이름 불러오는 함수
def searchItem(itemKey):
    item = item_df['name'][item_df['name'][item_df['key'] == itemKey].index[0]]
    return item 

In [26]:
# 영어이름의 스펠을 한글로 바꿔주는 함수
def translation_spell(eng):
    spell = spell_df['name'][spell_df['name'][spell_df['eng'] == eng].index[0]]
    return spell

In [27]:
# 룬키로 룬 이름 불러오는 함수
def searchRune(runeKey):
    rune = rune_df['name'][rune_df['name'][rune_df['key'] == runeKey].index[0]]
    return rune

In [28]:
# 태그 지우는 함수
def removeTags(strings):
    strings = re.sub('<.+?>', '', str(strings), 0).strip().replace('[','').replace(']','')
    return strings

In [29]:
# OP.GG 아이템이미지의 키를 구하는 함수 + 룬
def findItem(itemLink):
    items = re.findall(r'\d\d\d\d.png', str(itemLink))
    items = [item.strip('.png') for item in items]
    
    return items

In [30]:
# OP.GG 스킬이미지의 영문 이름을 구하는 함수
def findSkill(link):
    name = re.findall(r'spell/.*png', str(link))
    name = ''.join(name).strip('spell/').strip('.png')
    return name

In [31]:
lanes = ['TOP', 'JUNGLE', 'MID', 'ADC', 'SUPPORT']

In [32]:
for lane in lanes:
    top10 = soup.find(attrs={'class':f'tabItem champion-trend-tier-{lane}'}).select('.champion-index-table__name')[:10] # 10개만 가져오기
    top10 = removeTags(top10).replace("'",'')
    top10 = top10.split(', ')
    kor_champions = [translation_champion(champ) for champ in top10]
    kor_champions = ', '.join(kor_champions)
    print(f'{lane}의 최고티어 챔피언:',kor_champions)

TOP의 최고티어 챔피언: 카밀, 이렐리아, 피오라, 세트, 사일러스, 오공, 탐 켄치, 잭스, 비에고, 아트록스
JUNGLE의 최고티어 챔피언: 리 신, 신 짜오, 비에고, 엘리스, 카서스, 녹턴, 킨드레드, 샤코, 렉사이, 피들스틱
MID의 최고티어 챔피언: 카타리나, 르블랑, 탈론, 제드, 사일러스, 애니비아, 야스오, 판테온, 비에고, 이렐리아
ADC의 최고티어 챔피언: 직스, 이즈리얼, 베인, 애쉬, 사미라, 코그모, 스웨인, 징크스, 카이사, 진
SUPPORT의 최고티어 챔피언: 레오나, 룰루, 쓰레쉬, 세나, 제라스, 마오카이, 블리츠크랭크, 노틸러스, 질리언, 알리스타


### 원하는 챔피언의 기록 가져오기

In [33]:
# 샘플은 탑라인의 이렐리아
lane = 'top'
champion ='irelia'
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [34]:
win_rank = soup.select('.champion-stats-trend-rank')[0].find('b').text + soup.select('.champion-stats-trend-rank')[0].find('span').text
pick_rank = soup.select('.champion-stats-trend-rank')[1].find('b').text + soup.select('.champion-stats-trend-rank')[1].find('span').text
print('승률 순위:', win_rank, '픽률 순위:', pick_rank)

승률 순위: 27/55 픽률 순위: 1/55


### 원하는 챔피언의 추천 세팅 

- 스펠 , 스킬트리, 아이템 트리

In [35]:
for i in range(2):
    tmp = soup.select('ul.champion-stats__list')[i].find_all('img')[0] 
    tmp2 = soup.select('ul.champion-stats__list')[i].find_all('img')[1]
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[i].find('strong').text
    print('추천 스펠 : '+ translation_spell(findSkill(tmp)), translation_spell(findSkill(tmp2)), ' | 픽률 : '+ pickRate, '총표본 수 : ' + pickTotal, '승률 : '+ winRate)
skillSeq = []
for i in range(3):
    skill = findSkill(soup.select('ul.champion-stats__list')[2].find_all('img')[i*2])
    skillSeq.append(skill.lower().split(champion)[1])
pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('strong').text
pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick')[2].find('span').text
winRate = soup.select('.champion-overview__stats.champion-overview__stats--win')[2].find('strong').text
print('스킬 트리: ',' -> '.join(skillSeq).upper(),' | 픽률:', pickRate, '총표본 수 : ' + pickTotal, '승률:', winRate)    
for i in range(10):
    tmp = soup.select('td.champion-overview__data.champion-overview__border.champion-overview__border--first')[i].find_all('img')
    keys = findItem(tmp)
    items = [searchItem(int(key)) for key in keys]
    items = ', '.join(items)
    pickRate = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('strong').text
    pickTotal = soup.select('.champion-overview__stats.champion-overview__stats--pick.champion-overview__border')[i].find('span').text
    winRate = soup.select('.champion-overview__stats.champion-overview__stats--win.champion-overview__border')[i].find('strong').text
    if i < 2:
        msg = '시작아이템 :'
    elif i >= 2 and i < 7:
        msg = '추천 빌드 :'
    else:
        msg = '신발 :'
    print(msg + items, '| 픽률 : '+ pickRate,'총표본 수 : ' + pickTotal, '승률 : '+ winRate)

추천 스펠 : 점멸 순간이동  | 픽률 : 80.83% 총표본 수 : 56,171 승률 : 50.41%
추천 스펠 : 점멸 점화  | 픽률 : 17.99% 총표본 수 : 12,505 승률 : 49.78%
스킬 트리:  Q -> E -> W  | 픽률: 42.43% 총표본 수 : 13,721 승률: 62.22%
시작아이템 :도란의 검, 체력 물약 | 픽률 : 80.68% 총표본 수 : 54,936 승률 : 50.87%
시작아이템 :부패 물약 | 픽률 : 14.67% 총표본 수 : 9,990 승률 : 47.45%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 구인수의 격노검 | 픽률 : 20.5% 총표본 수 : 6,123 승률 : 62.57%
추천 빌드 :몰락한 왕의 검, 선혈포식자, 스테락의 도전 | 픽률 : 11.27% 총표본 수 : 3,366 승률 : 63.96%
추천 빌드 :몰락한 왕의 검, 신성한 파괴자, 스테락의 도전 | 픽률 : 10.57% 총표본 수 : 3,158 승률 : 62.6%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 마법사의 최후 | 픽률 : 8.82% 총표본 수 : 2,636 승률 : 62.52%
추천 빌드 :몰락한 왕의 검, 불멸의 철갑궁, 죽음의 무도 | 픽률 : 6.4% 총표본 수 : 1,911 승률 : 59.55%
신발 :판금 장화 | 픽률 : 69.13% 총표본 수 : 43,005 승률 : 51.98%
신발 :헤르메스의 발걸음 | 픽률 : 27.88% 총표본 수 : 17,343 승률 : 54.87%
신발 :명석함의 아이오니아 장화 | 픽률 : 2.19% 총표본 수 : 1,364 승률 : 61.14%


- 룬

In [36]:
mainRune = {'Precision' : '정밀', 'Domination' : '지배', 'Sorcery' : '마법', 'Resolve' : '결의', 'Inspiration' : '영감'}

In [37]:
# 첫번째 추천룬
total_info = soup.select_one('.tabItem.ChampionKeystoneRune-1')
rate_info = total_info.select('.champion-overview__stats.champion-overview__stats--pick')
rec_runes = soup.select('.champion-stats-summary-rune__name')
rune_info = soup.select('.champion-stats-summary-rune__rate')
eng_rune = rec_runes[0].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[0].find('strong').text
winRate = rune_info[0].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2): 
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
            k += 33
        link = total_info.find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 결의 | 픽률 : 73.64% 승률 : 50.54%
추천룬 | 픽률 : 23.98% 승률 : 48.75% 총 표본수 : 16,666
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
소생
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 8.45% 승률 : 51.69% 총 표본수 : 5,872
정밀
정복자
승전보
전설: 민첩함
최후의 저항
결의
뼈 방패
불굴의 의지
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9


In [38]:
# 두번째 추천 룬
rate_info = soup.select_one('.ChampionKeystoneRune-2').select('.champion-overview__stats.champion-overview__stats--pick')
eng_rune = rec_runes[1].text.split(' + ')
kor_rune = mainRune[eng_rune[0]] + ' + ' + mainRune[eng_rune[1]]
pickRate = rune_info[1].find('strong').text
winRate = rune_info[1].find_all('span')[-1].text
print(kor_rune, '| 픽률 : ' + pickRate, '승률 : ' + winRate)
for i in range(2):
    pickRate = rate_info[i].find('strong').text
    winRate= rate_info[i].find_all('strong')[-1].text
    pickTotal = rate_info[i].find_all('span')[1].text
    print('추천룬 | 픽률 : ' + pickRate, '승률 : ' + winRate, '총 표본수 : ' + pickTotal)
    for k in range(33):
        if i == 1:
                k += 33
        link = soup.select_one('.ChampionKeystoneRune-2').find_all('img')[k].attrs['src']
        if 'e_grayscale' in str(link):
            pass
        else:
            print(searchRune(int(''.join(findItem(link)))))

정밀 + 영감 | 픽률 : 15.34% 승률 : 48.63%
추천룬 | 픽률 : 3.6% 승률 : 47.12% 총 표본수 : 2,504
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
방어력 + 9
추천룬 | 픽률 : 1.54% 승률 : 46.35% 총 표본수 : 1,068
정밀
정복자
승전보
전설: 민첩함
최후의 저항
영감
비스킷 배달
시간 왜곡 물약
공격속도 + 10%
적응형 능력치 + 9
마법저항력 + 8


### 매치업

In [39]:
url = f'https://www.op.gg/champion/{champion}/statistics/{lane}/matchup'
html = requests.get(url).text
soup = BeautifulSoup(html, 'html.parser')

In [42]:
for i in range(30):
    champ = soup.select('.champion-matchup-list__champion')[i].find('span').text
    winRate = soup.select('.champion-matchup-list__champion')[i].find_all('span')[1].string
    winRate = [item.strip() for item in winRate if str(item)]
    winRate = ''.join(winRate)
    play_rate = soup.select('.champion-matchup-list__totalplayed')[i].find('span').text
    play_num = soup.select('.champion-matchup-list__totalplayed')[i].find('small').text
    print(f'{translation_champion(champion)} vs {translation_champion(champ)} 상대승률 :',winRate)
    print(f'통계량: {play_num} 통계 비율 {play_rate}')

이렐리아 vs 세트 상대승률 : 49.69%
통계량: 27,259 통계 비율 8.12%
이렐리아 vs 피오라 상대승률 : 48.57%
통계량: 22,298 통계 비율 6.64%
이렐리아 vs 사일러스 상대승률 : 51.47%
통계량: 18,353 통계 비율 5.47%
이렐리아 vs 카밀 상대승률 : 49.94%
통계량: 13,387 통계 비율 3.99%
이렐리아 vs 레넥톤 상대승률 : 52.51%
통계량: 12,769 통계 비율 3.80%
이렐리아 vs 잭스 상대승률 : 48.02%
통계량: 12,622 통계 비율 3.76%
이렐리아 vs 말파이트 상대승률 : 51.27%
통계량: 12,123 통계 비율 3.61%
이렐리아 vs 아트록스 상대승률 : 54.33%
통계량: 11,211 통계 비율 3.34%
이렐리아 vs 가렌 상대승률 : 48.06%
통계량: 11,006 통계 비율 3.28%
이렐리아 vs 그웬 상대승률 : 54.13%
통계량: 10,205 통계 비율 3.04%
이렐리아 vs 제이스 상대승률 : 56.55%
통계량: 9,211 통계 비율 2.74%
이렐리아 vs 리븐 상대승률 : 49.55%
통계량: 8,757 통계 비율 2.61%
이렐리아 vs 다리우스 상대승률 : 53.45%
통계량: 8,715 통계 비율 2.60%
이렐리아 vs 아칼리 상대승률 : 55.83%
통계량: 7,380 통계 비율 2.20%
이렐리아 vs 탐 켄치 상대승률 : 47.42%
통계량: 7,215 통계 비율 2.15%
이렐리아 vs 오공 상대승률 : 47.8%
통계량: 6,956 통계 비율 2.07%
이렐리아 vs 쉔 상대승률 : 48.79%
통계량: 6,645 통계 비율 1.98%
이렐리아 vs 오른 상대승률 : 50.01%
통계량: 6,323 통계 비율 1.88%
이렐리아 vs 볼리베어 상대승률 : 50.48%
통계량: 6,080 통계 비율 1.81%
이렐리아 vs 베인 상대승률 : 52.65%
통계량: 6,000 통계 비율 1.79%
이렐리아 vs 모데카이저 상대